In [1]:
from inference import SparseModel

/home/generall/projects/vector_search/tmp/msmacro/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SparseModel()

In [3]:
QDRANT_URL='http://localhost:6333'
QDRANT_API_KEY=''
COLLECTION_NAME="fiqa"

In [4]:
from qdrant_client import QdrantClient, models
import math

2024-03-17 23:16:17.033 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.


In [5]:
def rescore_vector(vector: dict) -> dict:
    new_vector = {}

    sorted_vector = sorted(vector.items(), key=lambda x: x[1], reverse=True)

    for num, (idx, _value) in enumerate(sorted_vector):
        new_vector[idx] = 1. # math.log(1./(num + 1) + 1.) # * value
    return new_vector


In [6]:
def conver_sparse_vector(sparse_vector: dict) -> models.SparseVector:
    indices = []
    values = []

    for (idx, value) in sparse_vector.items():
        indices.append(int(idx))
        values.append(value)

    return models.SparseVector(
        indices=indices,
        values=values
    )

In [7]:
client = QdrantClient(QDRANT_URL, api_key=QDRANT_API_KEY)

/home/generall/projects/vector_search/tmp/msmacro/venv/lib/python3.11/site-packages/qdrant_client/qdrant_remote.py:122: UserWarning: Api key is used with unsecure connection.
  warnings.warn("Api key is used with unsecure connection.")


In [12]:
query = "What are the consequences of IRS “reclassification” on both employer and employee?"
sparse_vector = rescore_vector(next(model.encode([query])))
sparse_vector_qdrant = conver_sparse_vector(sparse_vector)

limit = 10
result = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=models.NamedSparseVector(
        name="attention",
        vector=sparse_vector_qdrant
    ),
    with_payload=True,
    with_vectors=True,
    limit=limit
)

In [13]:
for idx, hit in enumerate(result):
    print(idx, hit)

0 id=1846 version=86 score=2.7154229 payload={'id': '19306', 'text': '"Even if your employer decides not to include the HSA contributions in Box 12, the IRS will still be informed how much went into your HSA when the form 5498-SA gets filed. So you don\'t need to worry about the IRS; they\'ll get the information they want.  As for you, if you already know how much the ""employer contributions"" (both what the employer contributed and what you contributed through payroll deduction) were, and you know how much you contributed directly, then once you get your form 1099-SA you\'ll have all the information you need to complete your tax return."'} vector={'attention': SparseVector(indices=[1000, 1005, 1006, 1007, 1010, 1011, 1012, 1025, 1056, 1996, 1998, 1999, 2000, 2004, 2005, 2017, 2020, 2022, 2025, 2027, 2031, 2035, 2043, 2046, 2050, 2054, 2055, 2059, 2061, 2065, 2083, 2097, 2113, 2115, 2119, 2123, 2129, 2130, 2131, 2139, 2145, 2172, 2215, 2222, 2253, 2260, 2320, 2342, 2421, 2433, 2525, 2

In [10]:
for (idx, value) in zip(sparse_vector_qdrant.indices, sparse_vector_qdrant.values):
    print(f"{model.invert_vocab[idx]}: {value}")

irs: 1.0
##lass: 1.0
employee: 1.0
employer: 1.0
##ification: 1.0
rec: 1.0
?: 1.0
the: 1.0
”: 1.0
“: 1.0
of: 1.0
consequences: 1.0
and: 1.0
are: 1.0
both: 1.0
on: 1.0
what: 1.0


In [11]:
result_sparse_vector = result[19].vector['attention']

for (idx, value) in zip(result_sparse_vector.indices, result_sparse_vector.values):
    if idx in sparse_vector:
        print(f"{model.invert_vocab[idx]}: {value} x {sparse_vector[idx]} = {value * sparse_vector[idx]}")
    else:
        print(f"{model.invert_vocab[idx]}: {value}")


": 0.2535534
,: 0.0070898435
-: 0.040873792
.: 0.0012019611
/: 0.16037752
:: 0.04982328
i: 0.020368716
the: 0.0051598963 x 1.0 = 0.0051598963
to: 0.0052111023
is: 0.009334377
for: 0.011726985
it: 0.035170324
an: 0.050727665
this: 0.018753069
after: 0.048385985
so: 0.03519337
more: 0.025586877
do: 0.056796554
such: 0.09250462
how: 0.040951286
##l: 0.1334535
us: 0.33508387
found: 0.09381019
without: 0.19356108
might: 0.065580584
am: 0.08188124
##9: 0.08183294
##0: 0.07611996
attempt: 0.1667324
seems: 0.11852843
tax: 0.17178875
roll: 1.0714914
funds: 0.81186086
impossible: 0.44882402
searching: 0.1548299
treat: 0.14849602
www: 0.06165745
##over: 0.56259817
consequences: 0.31056586 x 1.0 = 0.31056586
pub: 0.30211294
pdf: 0.2938165
https: 0.090242006
##80: 0.1379305
gov: 0.09503207
ut: 0.33013585
irs: 1.5292747 x 1.0 = 1.5292747
##200: 0.23212415


In [82]:
query = "Is math an art or a science?"
sv = next(model.encode([query]))
sv = conver_sparse_vector(sv)

for (idx, value) in zip(sv.indices, sv.values):
    print(f"{model.invert_vocab[idx]}: {value}")


is: 0.027513518929481506
math: 0.2162715643644333
an: 0.02588239312171936
art: 0.1304435133934021
or: 0.02149391360580921
a: 0.021214833483099937
science: 0.11642558127641678
?: 0.028545841574668884


In [40]:
import math

In [52]:
math.log(1/20 + 1)

0.04879016416943205

In [112]:
a = [1,3,2]

sorted(a, reverse=True)

a

[1, 3, 2]